# Samples 09-semantic_kernel_with_memory_kernel_plugin

Source : <https://github.com/microsoft/kernel-memory/blob/main/examples/003-dotnet-SemanticKernel-plugin/Program.cs>

In [2]:
// Load some helper functions, e.g. to load values from settings.json
#!import config/Settings.cs

In [4]:
// Import Semantic Kernel
#r "nuget: Microsoft.SemanticKernel"
#r "nuget: Microsoft.KernelMemory.WebClient"
#r "nuget: Microsoft.KernelMemory.SemanticKernelPlugin"

Installed Packages Microsoft.KernelMemory.SemanticKernelPlugin, 0.62.240605.1 Microsoft.KernelMemory.WebClient, 0.62.240605.1 Microsoft.SemanticKernel, 1.14.1

## Instantiate the kernel

In [5]:
using Microsoft.SemanticKernel;
using Kernel = Microsoft.SemanticKernel.Kernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.KernelMemory;
using Microsoft.SemanticKernel.ChatCompletion;



In [6]:
// Configure AI service credentials used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

Console.WriteLine($"Use Azure Open AI : {useAzureOpenAI}");
Console.WriteLine($"Model : {model}");
Console.WriteLine($"Azure EndPoint : {azureEndpoint}");
// Console.WriteLine($"API Key : {apiKey}");

//Create Kernel builder
var builder = Kernel.CreateBuilder();

if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);

var kernel = builder.Build();

Use Azure Open AI : True
Model : gpt-35-turbo-16k
Azure EndPoint : https://oai-ygo74-switzerland.openai.azure.com/


## Create prompt function

In [22]:
var promptOptions = new OpenAIPromptExecutionSettings {
    ChatSystemPrompt = "Answer or say \"I don't know\".",
    MaxTokens = 8191,
    Temperature = 0,
    TopP = 0
};


var skPrompt = """
Question: {{$input}}
Tool call result: {{memory.ask $input index='assurances'}}
If the answer is empty say "I don't know", otherwise reply with a preview of the answer, truncated to 15 words.
""";

var myFunction = kernel.CreateFunctionFromPrompt(skPrompt, promptOptions);

## Add memory plugin

In [8]:
var s_memory = new MemoryWebClient("http://127.0.0.1:9001/");

var memoryPlugin = kernel.ImportPluginFromObject(new MemoryPlugin(s_memory, waitForIngestionToComplete: true), "memory");

In [24]:
var question = "Quels sont les territoires et pays exclus de la couverture de l'assurance?";
Console.WriteLine($"Question: {question}");

var answer = await myFunction.InvokeAsync(kernel, question);
Console.WriteLine($"\nAnswer: {answer}\n\n  Sources:\n");


Question: Quels sont les territoires et pays exclus de la couverture de l'assurance?

Answer: Les territoires et pays exclus de la couverture de l'assurance sont les suivants: Afghanistan, Biélorussie, Birmanie/Myanmar, région de Crimée et r.

  Sources:



In [ ]:
## Chat

In [10]:
Console.WriteLine("# Starting chat...");
var chatService = kernel.GetRequiredService<IChatCompletionService>();


# Starting chat...


In [17]:
// Chat setup
var systemPrompt = """
                    You are a helpful assistant replying to user questions using information from your memory.
                    Reply very briefly and concisely, get to the point immediately. Don't provide long explanations unless necessary.
                    Sometimes you don't have relevant memories so you reply saying you don't know, don't have the information.
                    The topic of the conversation is assurance to a USA roadtrip.
                    """;

var chatHistory = new ChatHistory(systemPrompt);

// Start the chat
var assistantMessage = "Hello, how can I help?";
Console.WriteLine($"Copilot> {assistantMessage}\n");
chatHistory.AddAssistantMessage(assistantMessage);



Copilot> Hello, how can I help?



In [25]:
//
// Infinite chat loop
var reply = new StringBuilder();

// Get user message (retry if the user enters an empty string)
Console.Write("You> ");
var question = "Qu'est-ce qui n'est pas couvert par l'assurance?";

var promptTemplateFactory = new KernelPromptTemplateFactory();
var arguments = new KernelArguments
{
    ["input"] = question
};

string userMessage = await promptTemplateFactory.Create(new PromptTemplateConfig(skPrompt)).RenderAsync(kernel, arguments);
chatHistory.AddUserMessage(userMessage);

// Generate the next chat message, stream the response
Console.Write("\nCopilot> ");
reply.Clear();

await foreach (StreamingChatMessageContent stream in chatService.GetStreamingChatMessageContentsAsync(chatHistory))
{
    Console.Write(stream.Content);
    reply.Append(stream.Content);
}

chatHistory.AddAssistantMessage(reply.ToString());
Console.WriteLine("\n");



You> 
Copilot> Voici ce qui n'est pas couvert par l'assurance : guerres, catastrophes naturelles, usage de substances non médicales, etc.

